<a href="https://colab.research.google.com/github/mehlmanmichael/Coursera_Capstone/blob/main/Week_2_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup: Import libraries and connect to Foursquare API

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values
from IPython.display import Image 
from IPython.core.display import HTML 
import folium # plotting library
import json
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
#!pip install geocoder #Was trying to use geocoder but kept returning 'none' so I won't install it
#import geocoder

In [2]:
# Foursquare API credentials are stored on my google drive in a .py file.  Load them here.
from google.colab import drive
drive.mount('/content/drive')
import os
import sys
libpath = '/content/drive/My Drive/Programming/Python/Coursera Data Science Capstone'
sys.path.append(libpath)
import fsa
drive.flush_and_unmount()
#print(fsa.CLIENT_ID+fsa.CLIENT_SECRET+fsa.ACCESS_TOKEN)
VERSION = '20180604'
LIMIT = 100

Mounted at /content/drive


# Question 1: Scrape wikipedia and clean up dataframe

In [14]:
soup = BeautifulSoup(requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text,'lxml')
table = soup.find('table',{'class':'wikitable sortable'})
df = pd.read_html(str(table))[0] #create the df
df = df[df['Borough'] != 'Not assigned'] #get rid of not assigned boroughs
df.rename(columns = {'Neighbourhood':'Neighborhood'}, inplace=True) #spell neighborhood in the american fashion so I avoid mistakes later!
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned', df['Borough'], df['Neighborhood']) #borough but not neighborhood then neighborhood = borough
df = df.reset_index(drop=True)
print(df.head())
print(df.shape)

  Postal Code           Borough                                 Neighborhood
0         M3A        North York                                    Parkwoods
1         M4A        North York                             Victoria Village
2         M5A  Downtown Toronto                    Regent Park, Harbourfront
3         M6A        North York             Lawrence Manor, Lawrence Heights
4         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
(103, 3)


# Question 2: Add lat and long info

First I tried to do this with geocoder (as below), but it would not return any lat/lon coords. So I will just use the csv as below.

In [ ]:
df['Lat'] = df['Lon'] = np.nan
for index, row in df.head().iterrows():
  coords = None
  while (coords is None):
    coords = geocoder.google('{}, Toronto, Ontario'.format(row['Postal Code'])).latlng
  print(coords)
# Going to just download the csv... geocoder is not repsonding.

In [15]:
pc = pd.read_csv('https://cocl.us/Geospatial_data')
df = df.join(pc.set_index('Postal Code'), on='Postal Code')
df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Cluster with Foursquare data, map, etc.!

Limit to borough's containing the word Toronto

In [30]:
df = df[df['Borough'].str.contains('Toronto')]
print(df.shape)

(39, 5)


Define a function to loop over all neighborhoods and get venues

In [89]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(fsa.CLIENT_ID, fsa.CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    return(nearby_venues)

Run the get venues function (but don't do this too often due to quotas!)

In [32]:
venues = getNearbyVenues(names=df['Neighborhood'], latitudes=df['Latitude'], longitudes=df['Longitude'])

Perform onehot encoding on the venues and average per neihborhood so we can feed it into the clustering algo

In [90]:
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")
onehot['Neighborhood'] = venues['Neighborhood'] 
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,...,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Berczy Park,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.017241,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.017241,0.034483,0.000000,0.000000,0.0,0.017241,0.017241,0.0,0.034483,0.0,0.0,0.017241,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,...,0.0,0.000000,0.034483,0.0,0.017241,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.017241,0.0,0.000000,0.0,0.017241,0.0,0.0,0.017241,0.0,0.0,0.017241,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.000000,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.045455,0.000000,0.045455,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.090909,0.000000,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.045455,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
2,"Business reply mail Processing Centre, South C...",0.058824,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.058824,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.058824,0.000000,...,0.0,0.000000,0.000000,0.0,0.000000,0.058824,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0,0.058824,0.058824,0.058824,0.117647,0.117647,0.117647,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.058824,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.058824,0.0,0.058824,0.0,0.000000,0.000000,0.000000,...,0.0,0.058824,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0
4,Central Bay Street,0.015873,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.015873,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.015873,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.031746,...,0.0,0.000000,0.015873,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.015873,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.015873,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.031746,0.0,0.0,0.0,0.0,0.0,0.0,0.015873,0.0,0.0,0.015873,0.0


Define a function to limit the number of most common venues per hood

In [91]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

* Choose a number of top venues (5)
* Choose a number of clusters (5)
* Create a new df with generic column titles for "top venues"
* Limit the top venues for each hood and populate them into the new df
* run clustering
* insert the cluster labels back into the neighborhood df

In [92]:
num_top_venues = 5
n_clusters = 5
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    columns.append('{} Most Common Venue'.format(ind+1))
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']
for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)
grouped_clustering = grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(grouped_clustering)#fit clusters
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

* Merge this new sorted venues df back into the original df so we have lat/long, etc.
* see if it looks right

In [93]:
merged = df
merged = merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
merged.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1 Most Common Venue,2 Most Common Venue,3 Most Common Venue,4 Most Common Venue,5 Most Common Venue
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Bakery,Park,Café,Pub
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,1,Coffee Shop,Sushi Restaurant,Yoga Studio,Bank,Beer Bar
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Café,Bubble Tea Shop,Hotel
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Café,Cocktail Bar,American Restaurant,Gym
19,M4E,East Toronto,The Beaches,43.676357,-79.293031,3,Pub,Health Food Store,Trail,Wine Shop,Dog Run


Plot it on a map!  Cluster 1 is very common.  
**Note:** map does not show up in github.  Click [here](https://colab.research.google.com/drive/1e8Nys-FYMJgIsP5F4lgbA0JLfCH_T6Qw?usp=sharing) for colab version with map.

In [94]:
geolocator = Nominatim(user_agent='explorer')
location = geolocator.geocode('Toronto, Ontario')
latitude = location.latitude
longitude = location.longitude
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)
import matplotlib.cm as cm
import matplotlib.colors as colors
x = np.arange(n_clusters)
ys = [i + x + (i*x)**2 for i in range(n_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
for lat, lon, poi, cluster in zip(merged['Latitude'], merged['Longitude'], merged['Neighborhood'], merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker([lat, lon], radius=5, popup=label, color=rainbow[cluster-1], fill=True, fill_color=rainbow[cluster-1], fill_opacity=0.7).add_to(map_clusters)
map_clusters

For good measure, print out the most common "top" venues in each hood to get a feel for them.  
* **Cluster 0 (red):** Grocery / park, kind of suburban
* **Cluster 1 (purple):** Shops, cafe's, restaurants... downtown vibe
* **Cluster 2 (blue):** Upscale, sushi, wine
* **Cluster 3 (teal):** Trendy with variety of cuisines
* **Cluster 4 (orange):** Different type of suburban (dog run, etc.)

In [95]:
for i in merged['Cluster Labels'].unique():
  print('Top venues in cluster '+str(i)+' are:')
  print(merged[merged['Cluster Labels'] == i].iloc[:, 6:].describe(include='all').loc['top'])

Top venues in cluster 1 are:
1 Most Common Venue    Coffee Shop
2 Most Common Venue           Café
3 Most Common Venue           Café
4 Most Common Venue     Restaurant
5 Most Common Venue            Pub
Name: top, dtype: object
Top venues in cluster 3 are:
1 Most Common Venue        Business Service
2 Most Common Venue       Health Food Store
3 Most Common Venue                   Trail
4 Most Common Venue           Deli / Bodega
5 Most Common Venue    Ethiopian Restaurant
Name: top, dtype: object
Top venues in cluster 0 are:
1 Most Common Venue             Grocery Store
2 Most Common Venue               Coffee Shop
3 Most Common Venue                      Park
4 Most Common Venue             Grocery Store
5 Most Common Venue    Furniture / Home Store
Name: top, dtype: object
Top venues in cluster 4 are:
1 Most Common Venue      Home Service
2 Most Common Venue    Ice Cream Shop
3 Most Common Venue            Garden
4 Most Common Venue           Dog Run
5 Most Common Venue      Dessert